### Transformers, what can they do?

https://huggingface.co/course/chapter1/3?fw=pt

Transformer models are used to solve all kinds of NLP tasks. The following is a list of common NLP tasks, with some examples of each:

- **Classifying whole sentences**: Getting the sentiment of a review, detecting if an email is spam, determining if a sentence is grammatically correct or whether two sentences are logically related or not
- **Classifying each word in a sentence**: Identifying the grammatical components of a sentence (noun, verb, adjective), or the named entities (person, location, organization)
- **Generating text content**: Completing a prompt with auto-generated text, filling in the blanks in a text with masked words
- **Extracting an answer from a text**: Given a question and a context, extracting the answer to the question based on the information provided in the context
- **Generating a new sentence from an input text**: Translating a text into another language, summarizing a text

#### Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

#### Working with pipelines
The most basic object in the 🤗 Transformers library is the pipeline() function. It connects a model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer

#### Classification

In [3]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
classifier("movie had a good storyline but was very long and got boring towards the end")

[{'label': 'NEGATIVE', 'score': 0.9992792010307312}]

*We can even pass several sentences!*

In [6]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [7]:
classifier("the movie had a good storyline and could have been shorter duration")

[{'label': 'NEGATIVE', 'score': 0.8611639142036438}]

#### What is a pipeline?

By default, this pipeline selects a particular pretrained model that has been fine-tuned for sentiment analysis in English. The model is downloaded and cached when you create the classifier object. If you rerun the command, the cached model will be used instead and there is no need to download the model again.

There are three main steps involved when you pass some text to a pipeline:

- The text is preprocessed into a format the model can understand.
- The preprocessed inputs are passed to the model.
- The predictions of the model are post-processed, so you can make sense of them.

Some of the currently [available pipelines](https://huggingface.co/transformers/main_classes/pipelines.html) are:

- feature-extraction (get the vector representation of a text)
- fill-mask
- ner (named entity recognition)
- question-answering
- sentiment-analysis
- summarization
- text-generation
- translation
- zero-shot-classification

Let’s have a look at a few of these!

#### Zero-shot classification
- We’ll start by tackling a more challenging task where we need to classify texts that haven’t been labelled. This is a common scenario in real-world projects because annotating text is usually time-consuming and requires domain expertise.

- For this use case, the zero-shot-classification pipeline is very powerful: it allows you to specify which labels to use for the classification, so you don’t have to rely on the labels of the pretrained model.

- You’ve already seen how the model can classify a sentence as positive or negative using those two labels — but it can also classify the text using any other set of labels you like.

In [8]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [9]:
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445989489555359, 0.11197412759065628, 0.04342695698142052]}

In [10]:
classifier(
    "The markets were very jittery yesterday and the index closed 2 percent down",
    candidate_labels=["education", "politics", "business", "securities","stocks"],
)

{'sequence': 'The markets were very jittery yesterday and the index closed 2 percent down',
 'labels': ['securities', 'stocks', 'business', 'politics', 'education'],
 'scores': [0.5159638524055481,
  0.3615390956401825,
  0.10600214451551437,
  0.008256878703832626,
  0.008238023146986961]}

In [14]:
classifier(
    "It was an intersting game yesterday and India outplayed South Africa in batting and bowling",
    candidate_labels=["education", "politics", "business","securities","stock"],
)

{'sequence': 'It was an intersting game yesterday and India outplayed South Africa in batting and bowling',
 'labels': ['education', 'politics', 'business'],
 'scores': [0.4372616410255432, 0.310377836227417, 0.2523605525493622]}

*This pipeline is called zero-shot because you don’t need to fine-tune the model on your data to use it. It can directly return probability scores for any list of labels you want!*

#### Text generation
- Now let’s see how to use a pipeline to generate some text. The main idea here is that you provide a prompt and the model will auto-complete it by generating the remaining text.
- This is similar to the predictive text feature that is found on many phones. Text generation involves randomness, so it’s normal if you don’t get the same results as shown below.

In [15]:
from transformers import pipeline

generator = pipeline("text-generation")


No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [18]:
generator("In this course, we will teach you how to")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to use these tools to get more experience in the fields of computer science, business design, and computer visualization. As an added bonus, we can teach you how to write a test-based Java application for'}]

*You can control how many different sequences are generated with the argument `num_return_sequences` and the total length of the output text with the argument `max_length`.*

In [19]:
generator("In this course, we will teach you how to", num_return_sequences=10, max_length=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to work with your hands, and use your knowledge to'},
 {'generated_text': 'In this course, we will teach you how to create the latest technology to create the coolest designs.'},
 {'generated_text': 'In this course, we will teach you how to take a basic English level before passing through to a'},
 {'generated_text': 'In this course, we will teach you how to easily make simple, light-weight recipes using simple'},
 {'generated_text': 'In this course, we will teach you how to write code without having to learn every single word.'},
 {'generated_text': 'In this course, we will teach you how to create an infinite number of real numbers in the order'},
 {'generated_text': 'In this course, we will teach you how to navigate and manage your virtual world using JavaScript objects and'},
 {'generated_text': 'In this course, we will teach you how to create an easy to use Python programmable programming language'},
 {'generated_text': 'In

#### Using any model from the Hub in a pipeline
- The previous examples used the default model for the task at hand, but you can also choose a particular model from the Hub to use in a pipeline for a specific task — say, text generation.
- Go to the [Model Hub](https://huggingface.co/models) and click on the corresponding tag on the left to display only the supported models for that task. You should get to a page like [this one](https://huggingface.co/models?pipeline_tag=text-generation).

Let’s try the [distilgpt2](https://huggingface.co/distilgpt2) model! Here’s how to load it in the same pipeline as before:

In [20]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")


In [21]:
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to become a leader in your leadership skills as a successful leader. Then we will teach you how to think'},
 {'generated_text': 'In this course, we will teach you how to make a change from the typical life you’ll think about. Each course will be geared toward'}]

You can refine your search for a model by clicking on the language tags, and pick a model that will generate text in another language. The Model Hub even contains checkpoints for multilingual models that support several languages.

*Once you select a model by clicking on it, you’ll see that there is a widget enabling you to try it directly online. This way you can quickly test the model’s capabilities before downloading it.*



#### The Inference API
All the models can be tested directly through your browser using the Inference API, which is available on the Hugging Face website. You can play with the model directly on this page by inputting custom text and watching the model process the input data.

#### Mask filling
The next pipeline you’ll try is fill-mask. The idea of this task is to fill in the blanks in a given text:

In [22]:
from transformers import pipeline

unmasker = pipeline("fill-mask")


No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [23]:
unmasker("This course will teach you all about <mask> models.", top_k=5)

[{'score': 0.19619806110858917,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052723944187164,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'},
 {'score': 0.03301795944571495,
  'token': 27930,
  'token_str': ' predictive',
  'sequence': 'This course will teach you all about predictive models.'},
 {'score': 0.031941577792167664,
  'token': 745,
  'token_str': ' building',
  'sequence': 'This course will teach you all about building models.'},
 {'score': 0.024522852152585983,
  'token': 3034,
  'token_str': ' computer',
  'sequence': 'This course will teach you all about computer models.'}]

- The top_k argument controls how many possibilities you want to be displayed
- Note that here the model fills in the special `<mask>` word, which is often referred to as a mask token. Other mask-filling models might have different mask tokens, so it’s always good to verify the proper mask word when exploring other models. One way to check it is by looking at the mask word used in the widget.
- **Try it out!** Search for the bert-base-cased model on the Hub and identify its mask word in the Inference API widget. What does this model predict for the sentence in our pipeline example above?

#### Named entity recognition
Named entity recognition (NER) is a task where the model has to find which parts of the input text correspond to entities such as persons, locations, or organizations. Let’s look at an example:

In [24]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [25]:
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

- We pass the option `grouped_entities=True` in the pipeline creation function to tell the pipeline to regroup together the parts of the sentence that correspond to the same entity: here the model correctly grouped “Hugging” and “Face” as a single organization, even though the name consists of multiple words
- **Try it out!** Search the Model Hub for a model able to do part-of-speech tagging (usually abbreviated as POS) in English. What does this model predict for the sentence in the example above?

#### Question answering
The question-answering pipeline answers questions using information from a given context:

In [26]:
from transformers import pipeline

question_answerer = pipeline("question-answering")


No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [27]:
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

{'score': 0.6949767470359802, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}

*Note that this pipeline works by extracting information from the provided context; it does not generate the answer.*

In [29]:
question_answerer(
    question="Which countries have managed to launch a spacecraft to moon?",
    context = '''The spacecraft's health is normal. The first orbit raising manoeuvre (Earthbound firing-1) was successfully performed at ISTRAC/ISRO, Bengaluru. Spacecraft is now in 41762kms x 173kms orbit," ISRO said.
ISRO on July 14 successfully launched the third edition of its Moon exploration programme from the Satish Dhawan Space Centre, Sriharikota, aimed at making a soft landing on the unexplored south pole of the Moon that would make India achieve a rare feat.
Only three countries, the United States, China and Russia, have managed to land on the lunar surface so far.''',
)

{'score': 0.562257707118988,
 'start': 484,
 'end': 515,
 'answer': 'United States, China and Russia'}

In [ ]:
question_answerer(
    question="Where is Hugging Face situated?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

#### Summarization
Summarization is the task of reducing a text into a shorter text while keeping all (or most) of the important aspects referenced in the text. Here’s an example:

In [30]:
from transformers import pipeline

summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [31]:
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

[{'summary_text': ' The number of engineering graduates in the United States has declined in recent years . China and India graduate six and eight times as many traditional engineers as the U.S. does . Rapidly developing economies such as China continue to encourage and advance the teaching of engineering . There are declining offerings in engineering subjects dealing with infrastructure, infrastructure, the environment, and related issues .'}]

In [ ]:
summarizer(
"""
The hybrid lifestyle that we have adopted especially after the pandemic has created a need for products that are not only fresh but also break away from the clutter. The essence of the need is also to find something that is affordable, has impactful features, and offers an excellent after-sales service. As a result of this movement, the Indian marketplace has evolved manifolds, today, it is being driven by young millennial consumers who seek quality and uniqueness. And rightfully so, they demand products that are SENSible yet SENSational. Which basically means a mix of style and functionality minus a huge dent in their pockets. Now to address that, a new brand SENS has come into the picture.
Keeping the needs of the smart device users of today in mind, SENS has introduced a wide portfolio of 100% made-in-India products including smartwatches, TWS earbuds, and neckbands targeting the Millennials, GenZ, and everyone who is willing to buy stylish yet high-performing connected wearables.
Loaded with features and some industry-first technologies, all SENS products come at affordable prices ranging between INR 699* to 3,099*.
SENS is backed by the renowned Jaina group which has more than a decade-long experience in the mobile, smartwatch, wearable, and consumer durable industry.
SENSational technology
SENS’ portfolio features some of the industry-first trademarked technologies for a superior consumer experience. One such technology is IIC™ (Intelligent Insta Connect) – a feature that lets consumers seamlessly pair their earbuds as soon as they open the lid. Another feature is SVVC™ (Smart Video Voice Connect) which significantly reduces latency in select TWS and neckbands and keeps the audio/video in seamless sync. SENS products will also come with a trademarked fast charging technology that will offer a quick charge facility to refuel the device to full capacity in record time. Along with that, the smartwatch portfolio comes featured with the latest technologies like AMOLED Display, BT calling, Orbiter, etc.

It's interesting that the brand has named all its products after renowned scientists and artists. With enticing names such as Edyson, MJ, and Einsteyn, SENS' products indeed make an impact with their higher recall value.
""")

**Like with text generation, you can specify a max_length or a min_length for the result.**

#### Translation
For translation, you can use a default model if you provide a language pair in the task name (such as "translation_en_to_fr"), but the easiest way is to pick the model you want to use on the [Model Hub](https://huggingface.co/models). Here we’ll try translating from French to English:

In [32]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [33]:
translator("Ce cours est produit par Hugging Face.")

[{'translation_text': 'This course is produced by Hugging Face.'}]

In [34]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

In [ ]:
text2 = """
Author and journalist Rahul Pandita lived through the exodus of the Kashmiri \
Pandit community in the 1990s, but says he’s still at the receiving end of \
abuse on social media for his views that run contrary to popular views on the \
subject. In this episode of The TOI podcast, he explains why the community is \
still bitter about its experience after decades, why a recent film is so \
popular among them and a very personal incident that showed how those times \
affected people till today. Why Kashmir Files is so popular
"""

In [35]:
# translator = pipeline("translation_en_to_de",
#                       model="Helsinki-NLP/opus-mt-en-de")
translator = pipeline("translation_en_to_hi",
                      model="Helsinki-NLP/opus-mt-en-hi")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

प्रिय एमिला, पिछले सप्ताह मैंने जर्मनी में आपके ऑनलाइन स्टोर से एक ऑपिश क्रिया आकृति का आदेश दिया. दुःख की बात है, जब मैंने पैकेज खोला था, तब मुझे लगा कि मैं मेगारोन की एक कार्य आकृति भेजा गया था! इसके बजाय, मैं आशा करता हूँ कि आप मेरी दुविधा को समझ सकते हैं. मुझे लगता है कि Megogigiociod के मामले के बारे में एक Meghatolialphy की मांग की है. मैं पिछले सप्ताह के बारे में एक Mephatolirighphphphphphphphphphphphphy की प्रतीक्षा कर रहे हैं. मैं अपनी सूची के बारे में बहुत जल्द ही सुन रहा हूँ. मुझे लगता है कि मैं एक बहुत जल्द ही अच्छी तरह से सुन रहा हूँ.


Like with text generation and summarization, you can specify a `max_length` or a `min_length` for the result.

**Try it out!** Search for translation models in other languages and try to translate the previous sentence into a few different languages.


#### Whats inside a pipeline?

In [ ]:
# Code when using a pipeline
# from transformers import pipeline
# classifier = pipeline("sentiment-analysis")

In [36]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")



##### Step 1: Tokenization

In [37]:
#step 1: tokenization
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [38]:
print(type(inputs))
print(inputs.keys())


<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'attention_mask'])


In [39]:
print(inputs['input_ids'])
print(inputs['attention_mask'])

tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1]])


##### Step 2: Model consumes input and gives prediction

In [40]:
#step 2: model consumes the input and gives prediction
with torch.no_grad():
    logits = model(**inputs).logits


##### Step 3: processing model's prediction and formatting in an understandable manner

In [41]:
#step 3: processing model's prediction and formatting in an understandable manner
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'POSITIVE'

In [42]:
predicted_class_id

1

In [43]:
model.config.id2label[0]

'NEGATIVE'

#### Another example

In [ ]:
inputs = tokenizer("good storyline but a very long movie", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

In [ ]:
predicted_class_id